### Setting up the stage!

In [1]:
import os
import pandas as pd
import datetime as dt

### User Input Required!

In [2]:
# Add the csv file name and location here!
# Get the file under DCA Bot > My Deals > Deals History > Export
source = "data/3commas-dca-deals-210101.csv"

### Other variables

In [3]:
# Creates empty dataframe
summary = pd.DataFrame()
summary.index.name = "Bot ID"

#TODO REVIEW ALL STATUSES
trade_status_closed = ["completed", "stop_loss_finished"]
trade_status_wins = ["completed"]
trade_status_losses = ["stop_loss_finished"]
trade_status_active = ["bought"] 

### Grabbing the data

In [4]:
data = pd.read_csv(source,
                   sep=";",
                   index_col="deal_id",
                   header=0,
                   parse_dates=['closed_at'])

### Raise an error if there's more than one market

In [5]:
# Check if the base asset is the same in all pairs 
base_pair = data["pair"].str.split("_").str[0].unique()
if len(base_pair) == 1:
    base_pair = base_pair[0]
else:
    raise("ERROR! 🙀 More than one market, not prepared for that!")

### Add features

In [6]:
# Add feature '% Profit" (bought volume x final profit)
data.loc[data["status"].isin(trade_status_closed), "% Profit"] = (data["final_profit"] * 100) / data["bought_volume"]

### Generate Report

In [7]:
for bot_id in data["bot_id"].unique():
    trades_total = len(data[(data["bot_id"] == bot_id)])
    trades_closed = len(data[(data["bot_id"] == bot_id) & (data["status"].isin(trade_status_closed))])
    trades_active = len(data[(data["bot_id"] == bot_id) & (data["status"].isin(trade_status_active))])
    trades_win = len(data[(data["bot_id"] == bot_id) & (data["status"].isin(trade_status_wins))])*100 / trades_closed
    trades_win = round(trades_win, 2)
    trades_win_avg = data[(data["bot_id"] == bot_id) & (data["status"].isin(trade_status_wins))]["final_profit"].mean()
    trades_losses = len(data[(data["bot_id"] == bot_id) & (data["status"].isin(trade_status_losses))])*100 / trades_closed
    trades_losses = round(trades_losses, 2)
    trades_duration_avg = data[data["bot_id"] == bot_id]["duration"].mean()
    trades_duration_avg = pd.to_datetime(trades_duration_avg, unit="s")
    trades_so_avg = data[data["bot_id"] == bot_id]["used_safety_orders"].mean()
    

    # Add data to Summary dataframe
    summary.loc[bot_id, "Bot Name"] = data[data["bot_id"] == bot_id]["bot"].unique()
    summary.loc[bot_id, "% AVG Profit"] = round(data[(data["bot_id"] == bot_id)]["% Profit"].mean(),2)
    profit_column_name = f"Profit ({base_pair})"
    summary.loc[bot_id, profit_column_name] = data[(data["bot_id"] == bot_id) & (~data["status"].isin(trade_status_active))]["final_profit"].sum()
    summary.loc[bot_id, "% Win"] = trades_win
    summary.loc[bot_id, "% Loss"] = trades_losses
    # summary.loc[bot_id, "Active Trades"] = trades_active
    summary.loc[bot_id, "Closed Trades"] = trades_closed
    summary.loc[bot_id, "AVG Trade Duration"] = trades_duration_avg
    summary.loc[bot_id, "AVG SO/Trade"] = trades_so_avg
    # TODO: AVG DEALS PER DAY/TIME THE BOT HAS BEEN ACTIVE // REQUIRES INFO IN THE SOURCE DATA
    
summary["AVG Trade Duration"] = summary["AVG Trade Duration"].dt.strftime("%H:%M:%S")
summary

,Bot Name,% AVG Profit,Profit (USDT),% Win,% Loss,Closed Trades,AVG Trade Duration,AVG SO/Trade
Bot ID,,,,,,,,
2145925,Gordon-Moderate,2.39,152.629137,100.00,0.00,39.0,03:34:25,0.769231
2145929,Gordon-Agressive,0.30,-30.753176,97.44,2.56,39.0,05:39:31,0.600000
2145913,Gordon-Conservative,-0.41,-60.023527,89.47,10.53,19.0,04:13:40,0.842105
2147462,Gordon-LEV-Conservative,1.35,29.542041,97.22,2.78,36.0,02:09:48,1.135135
2149831,Gordon-LEV-Agressive,-0.42,-60.457766,88.89,11.11,18.0,02:27:02,0.888889
2149682,Gordon-LEV-Moderate,-1.30,-154.772038,86.67,13.33,45.0,01:57:09,1.422222
